In [1]:
import os
os.chdir('../')
%pwd

'/home/xaris/Desktop/Projects/Chicken-Disease-Classification /Chicken-Disease-Classification-1'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    checkpoint_model_filepath: Path
    tensorboard_root_log_dir: Path

In [3]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(model_ckpt_dir), Path(config.tensorboard_root_log_dir)])


        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir)
        )
        
        return prepare_callbacks_config

In [5]:
# Create the components of the pipeline
import os
import urllib.request as request
import zipfile as ZipFile
import tensorflow as tf
import time

2025-10-10 12:23:09.618457: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-10 12:23:09.708719: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-10 12:23:11.517516: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [6]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callback(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(self.config.tensorboard_root_log_dir, f"tb_logs_at_{timestamp}")
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callback(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
        
    def get_tb_ckpt_callbacks(self):
        return [self._create_tb_callback, self._create_ckpt_callback]


In [7]:
# Execute the code to prepare the base model. Pipelines will be used to execute the code in a sequence.
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e 

[2025-10-10 16:29:26,461: INFO:common: yaml file: config/config.yaml loaded successfully]
[2025-10-10 16:29:26,466: INFO:common: yaml file: params.yaml loaded successfully]
[2025-10-10 16:29:26,467: INFO:common: created directory at: artifacts/]
[2025-10-10 16:29:26,469: INFO:common: created directory at: artifacts/prepare_callbacks/checkpoint_dir/model.h5]
[2025-10-10 16:29:26,471: INFO:common: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir]


In [8]:
import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
print(f"Timestamp: {timestamp}")

Timestamp: 2025-10-10-16-36-03
